## Grupo 2

visualizar histórico en un período de tiempo 
Histórico por día 
Histórico por mes
Histórico por año 
(pensar filtros: por planta, por tipo de material, por LE y RA)

Parte a coordinar con el grupo 3
Inputs= step temporal (día, mes, año) 
Output= Cantidad de material histórico sobre la base de datos total


In [1]:
# from google.colab import files
# uploaded = files.upload()
# for item in uploaded.keys():
#  print('Se subio "{}"'.format(item))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [3]:
pesajes_df=pd.read_csv("pesajes-30-07-2021-06-08-2021_anonimizado.csv", low_memory = False)
pesajes_df["fecha"] = pd.to_datetime(pesajes_df["fecha"],format="%Y-%m-%d")
# pesajes_df["año"] = pd.DatetimeIndex(pesajes_df.fecha).year
# pesajes_df["mes"] = pd.DatetimeIndex(pesajes_df.fecha).month
# pesajes_df["día"] = pd.DatetimeIndex(pesajes_df.fecha).day
pesajes_df.head(10)

fecha etapa  bolson  camion  peso cartonero legacyId    predio  \
0 2021-07-30    R8     2.0     NaN    24   COMPA40     2606  BARRACAS   
1 2021-07-30    R8     4.0     NaN    15   COMPA47     2608  BARRACAS   
2 2021-07-30    R8     4.0     NaN    21   COMPA47     2608  BARRACAS   
3 2021-07-30    R8     4.0     NaN    24   COMPA47     2608  BARRACAS   
4 2021-07-30    R8     5.0     NaN    35  COMPA112     6387  BARRACAS   
5 2021-07-30    R8    10.0     NaN    27  COMPA295     4860  BARRACAS   
6 2021-07-30    R8    10.0     NaN    32  COMPA295     4860  BARRACAS   
7 2021-07-30    R8    12.0     NaN    28  COMPA297     2619  BARRACAS   
8 2021-07-30    R8    15.0     NaN    73  COMPA731     2626  BARRACAS   
9 2021-07-30    R8    15.0     NaN    33  COMPA731     2626  BARRACAS   

   material  
0  Mezcla B  
1  Mezcla B  
2  Mezcla B  
3  Mezcla B  
4  Mezcla B  
5  Mezcla B  
6  Mezcla B  
7  Mezcla B  
8  Mezcla B  
9  Mezcla B

In [4]:
def pesos_historico_predios(data):
    '''
    Grafica fecha contra peso (promedio o suma total) entregado ese día agrupando según el predio.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["fecha", "predio", "material"], as_index = False).sum()    # Para sacar la info de materiales, sacar "material"
    fig = px.bar(df, x="fecha", y="peso", color="predio", barmode = "group", hover_data=["material"])
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    # Ignora fin de semana
    return fig

In [5]:
def pesos_historico_materiales(data):
    '''
    Grafica fecha contra peso (promedio o suma total) entregado ese día agrupando según el predio.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["fecha", "material"], as_index = False).sum()
    fig = px.bar(df, x="fecha", y="peso", color="material", barmode = "group")
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    # Ignora fin de semana
    return fig

In [6]:
def pesos_historico_promedio(data):
    '''
    Prueba. Grafica el "rolling average" (en cada dia muestra el promedio de los últimos 7 dias). 
    '''
    df = data.groupby(by=["fecha"], as_index = False).sum()
    df = df.rolling("7d", on = "fecha").sum()
    df["peso"] = df["peso"] / 7
    fig = px.line(df, x = "fecha", y = "peso")
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])    # Ignora fin de semana
    return fig

In [7]:
def torta(data):
    '''
    Realiza un gráfico de torta del peso por material.
    Devuelve la figura de plotly conteniendo el gráfico.
    '''
    df = data.groupby(by=["material"], as_index = False).sum()
    fig = px.pie(df, values="peso", names="material", title='')
    fig.update_traces(hoverinfo="label+percent", textfont_size=14,
                      textinfo="percent", marker=dict(line=dict(color="#FFFFFF",width=0.1)),
                      textposition="auto")
    return fig

In [8]:
# Ejemplo de Métrica
def centro_principal(data):
    '''
    Devuelve una lista que contiene el nombre del predio que recolectó un mayor peso total.
    Si hay empate, devuelve multiples nombres.
    '''
    df = data.groupby(by=["predio"], as_index = False).sum()
    res = df[df.peso == df.peso.max()]["predio"].to_list()
    return res


In [9]:
pesos_historico_predios(pesajes_df).show()
pesos_historico_materiales(pesajes_df).show()
pesos_historico_promedio(pesajes_df).show()
torta(pesajes_df).show()
print(f"El centro que más material reunió (en peso total) fue: {centro_principal(pesajes_df)[0]}")

El centro que más material reunió (en peso total) fue: BARRACAS
